In [1]:
from pycocotools.coco import COCO
import tensorflow as tf
import os


In [2]:
#resize images

img_folder = 'C:/COCO/coco2017/train2017/'
target_size = (224,224) #common size

def resize_image(image_path, target_size=(224, 224)):
    image = tf.io.read_file(image_path)  #load the image as binary data
    image = tf.image.decode_jpeg(image, channels=3)  #decode the image as RGB
    image = tf.image.resize(image, target_size)
    return image

for img_file in os.listdir(img_folder):
    if img_file.endswith('.jpg'):
        try:
            img_path = os.path.join(img_folder,img_file)
            resized = resize_image(img_path)
            #need to cast images back into uint8 format because tensorflow auto converts images to float32 for smooth transformations
            #changes tensorflow tensor array to numpy array for easy python processing
            recasted = tf.cast(resized, tf.uint8).numpy()
            #saves numpy array into image format
            tf.keras.preprocessing.image.save_img(img_path, recasted)
            
        except Exception as e:
            print(f"Error processing {img_file} :{e}")

In [3]:
#readjust annotations after resizing

ann_file = 'C:/COCO/coco2017/annotations/instances_train2017.json'
coco = COCO(ann_file)

#Below is just to demonstrate if you want to load categories
#categories = coco.loadCats(coco.getCatIds())
#print("Categories:",categories)

img_ids = coco.getImgIds()
for image_id in img_ids:
    ann_ids = coco.getAnnIds(imgIds = image_id)
    annotations = coco.loadAnns(ann_ids)
    
    img_info = coco.loadImgs(image_id)[0]
    original_width = img_info['width']
    original_height = img_info['height']
    
    x_scale = 224 / original_width
    y_scale = 224 / original_height
    
    for ann in annotations:
        bbox = ann['bbox']
        new_x = bbox[0] * x_scale
        new_y = bbox[1] * y_scale
        new_width = bbox[2] * x_scale
        new_height = bbox[3] * y_scale
        ann['bbox'] = [new_x,new_y,new_width,new_height]

loading annotations into memory...
Done (t=13.76s)
creating index...
index created!
